---
## Projectr : 하루시작 지하철 혼잡도 분석 02 
## Description : 
    - 서울 교통공 역별 일별 시간대별 승하차인원 정보
    [source link] (https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do#)
    
## Author : Forrest Dpark (분석 담당)
## Date : 2024.05.31 ~
## Detail : 
## Update: 
    - 2024.06.02 pdg : 일별 데이터 정제 
        - 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.


In [6]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [174]:
# Data loading 
subway = pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20221231.csv", encoding='euc-kr')
subway.head()

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,1,2022-01-01,1,150,서울역,승차,120,137,211,439,...,1566,1686,1591,1358,1062,899,1327,814,234,NaN
1,2,2022-01-01,1,150,서울역,하차,113,560,617,910,...,1329,1251,1126,884,764,654,728,416,131,NaN
2,3,2022-01-01,1,151,시청,승차,38,66,101,139,...,474,550,672,528,420,434,491,232,38,NaN
3,4,2022-01-01,1,151,시청,하차,31,195,224,380,...,408,377,354,213,131,98,137,61,24,NaN
4,5,2022-01-01,1,152,종각,승차,44,71,86,158,...,889,964,1024,803,855,1099,1209,255,62,NaN


In [175]:
def dataInfoProcessing(df, replace_Nan=False):
    ''' 
    # Fucntion Description :  Data frame 의 정제해야할 부분을 체크해주는 함수 
    # Date : 2024.06.02 
    # Author : Forrest D Park 
    # update : 
        * 2024.06.02 by pdg: 일별 데이터 정제 
            - 데이터에 null 이 있음을 발견, data 정제 함수 update 
            - 함수에서 replace_Nan 아규 멘트 받아서 true 일경우 nan 을 0 으로 대체 하게 만듬. 


    '''
    
    print("\n1. Data colum numbers : ",len(df.columns))
    #print(subway.columns)
    #print(subway.info())
    null_message =f"총 {df.isnull().sum().sum()}개의 null 이 있습니다!" if df.isnull().sum().sum() else "Null 없는 clean data!"
    print("\n2. null ceck 결과",null_message)
    ### Null 이 있는 칼럼 추출
    haveNullColumn =[]
    for idx, col in enumerate(df.columns):
        if df[f"{col}"].isnull().sum():
            print(f"   => {idx}번째.[{col}]컬럼 : ",f"null {df[f'{col}'].isnull().sum()} 개,\t not null {df[f'{col}'].notnull().sum()} 개")
            ## Null data fill
    if replace_Nan : ## nan 을 0 으로 대체 
        df[col].fillna(value=0, inplace=True)  
    
    print("\n3. Column  Information (중복체크)")
    print( "\tidx.columName |\t\t\t\t |Colum Info(dtype)|** ")
    print( "\t","--"*len("columIdx |\t\t\t\t **|Col(dtype)|** "))
    for idx, col in enumerate(df.dtypes.keys()):
        if idx< 7:
            if len(f"\t{idx}.[{col}({df.dtypes[col]})]:")<20:
                print(f"\t{idx}.[{col}({df.dtypes[col]})]:",\
                    f"{len(df[col].unique())}/{len(df[col])} [uniq/raw]", sep=" \t\t\t")
            else:
                    print(f"\t{idx}.[{col}({df.dtypes[col]})]:",\
                    f"{len(df[col].unique())}/{len(df[col])} [uniq/raw]", sep=" \t\t")

    else: 
        print(f"\t ...etc (추가로 {len(df.dtypes.keys())-5}개의 칼럼이 있습니다 )")
   
              
dataInfoProcessing(subway)

# subway.info(show_counts=False)
# subway.value_counts('수송일자')






1. Data colum numbers :  26

2. null ceck 결과 총 82346개의 null 이 있습니다!
   => 25번째.[24시이후]컬럼 :  null 82346 개,	 not null 116734 개

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


> 25번째 칼럼인 24시 이후 데이터의 null 은 운행을 하지 않아서 null 인가?

In [156]:
interesting_colnumbers = [i for i in range(1,7)] + [-2,-1]
print(interesting_colnumbers)
subway[subway['24시이후'].isnull()].iloc[:,interesting_colnumbers].head()

[1, 2, 3, 4, 5, 6, -2, -1]


,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,23-24시간대,24시이후
0,2022-01-01,1,150,서울역,승차,120,234,NaN
1,2022-01-01,1,150,서울역,하차,113,131,NaN
2,2022-01-01,1,151,시청,승차,38,38,NaN
3,2022-01-01,1,151,시청,하차,31,24,NaN
4,2022-01-01,1,152,종각,승차,44,62,NaN


In [161]:
havenulldate = subway[subway['24시이후'].isnull()].iloc[:,a]
havenulldate['수송일자'].unique()[[1,2,-2,-1]]
# 2022 년 1 월 1일부터  5월 31 일까지 는 12시 이후 영업을 안한 모양.. -> 코로나 단축운행!! 


array(['2022-01-02', '2022-01-03', '2022-05-30', '2022-05-31'],
      dtype=object)

> Nan 을 0 으로 대체 

In [177]:

dataInfoProcessing(subway,replace_Nan=True)


1. Data colum numbers :  26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


>  해당 데이터는 8개의 호선이 있고 역번호는 558개, 역명은 244 개가 있다.. 왜?  역번호와 역 명은 1:1 이어야 하는게 아닌가?

In [190]:
## 역명 조회
# 지하철 역 테이블 과 역번호 를 따로 만들어서 관리하도록 하자. csv 저장 필요 
for i in subway['역명'].unique():
    for 순서, 역명 in enumerate(subway['역명']):
        if 역명==i :
            print(subway['호선'][순서],f"호선 {i} 역")
            break


1 호선 서울역 역
1 호선 시청 역
1 호선 종각 역
1 호선 종로3가 역
1 호선 종로5가 역
1 호선 동대문 역
1 호선 신설동 역
1 호선 제기동 역
1 호선 청량리(서울시립대입구) 역
1 호선 동묘앞 역
2 호선 을지로입구 역
2 호선 을지로3가 역
2 호선 을지로4가 역
2 호선 동대문역사문화공원(DDP) 역
2 호선 신당 역
2 호선 상왕십리 역
2 호선 왕십리(성동구청) 역
2 호선 한양대 역
2 호선 뚝섬 역
2 호선 성수 역
2 호선 건대입구 역
2 호선 구의(광진구청) 역
2 호선 강변(동서울터미널) 역
2 호선 잠실나루 역
2 호선 잠실(송파구청) 역
2 호선 잠실새내 역
2 호선 종합운동장 역
2 호선 삼성(무역센터) 역
2 호선 선릉 역
2 호선 역삼 역
2 호선 강남 역
2 호선 교대(법원.검찰청) 역
2 호선 서초 역
2 호선 방배 역
2 호선 사당 역
2 호선 낙성대(강감찬) 역
2 호선 서울대입구(관악구청) 역
2 호선 봉천 역
2 호선 신림 역
2 호선 신대방 역
2 호선 구로디지털단지 역
2 호선 대림(구로구청) 역
2 호선 신도림 역
2 호선 문래 역
2 호선 영등포구청 역
2 호선 당산 역
2 호선 합정 역
2 호선 홍대입구 역
2 호선 신촌 역
2 호선 이대 역
2 호선 아현 역
2 호선 충정로(경기대입구) 역
2 호선 용답 역
2 호선 신답 역
2 호선 도림천 역
2 호선 양천구청 역
2 호선 신정네거리 역
2 호선 용두(동대문구청) 역
3 호선 지축 역
3 호선 구파발 역
3 호선 연신내 역
3 호선 불광 역
3 호선 녹번 역
3 호선 홍제 역
3 호선 무악재 역
3 호선 독립문 역
3 호선 경복궁(정부서울청사) 역
3 호선 안국 역
3 호선 동대입구 역
3 호선 약수 역
3 호선 금호 역
3 호선 옥수 역
3 호선 압구정 역
3 호선 신사 역
3 호선 잠원 역
3 호선 고속터미널 역
3 호선 남부터미널(예술의전당) 역
3 호선 양재(서초구청) 역
3 호선 매봉 역
3 호선 도곡 역
3 호선 대치 역
3 호선 학여울